- Complexity: number of firms for which analyst i supplied at least one forecast during the first 11 months of year t OR number of two-digit SICs for which analyst i supplied at least one forecast during the first 11 months of year t minus the average number of two-digit SICs followed by an analyst following firm j at time t.

- Forecasting ability/experience: number of years for which analyst i supplied at least one forecast during the first 11 months of the year through year t